# Cox model with Spark — end-to-end tutorial

Эта тетрадка показывает минимальный рабочий цикл: обучение, продление baseline, инференс, сохранение в единый CSV и создание модели только из конфигурации.

In [1]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

from spark_lifelines_cox.model import SparkCoxPHByType

## 1. Стартуем Spark

In [2]:
spark = SparkSession.builder.master('local[*]').appName('cox_tutorial').getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/12/17 12:00:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2. Синтетические данные

In [3]:
rng = np.random.default_rng(7)
pdf = pd.DataFrame({
    'type': np.where(rng.random(120) > 0.5, 'A', 'B'),
    'duration': rng.exponential(scale=6, size=120),
    'event': rng.binomial(1, 0.75, size=120),
    'x': rng.normal(size=120),
})
sdf = spark.createDataFrame(pdf)
sdf.show(5)

/root/.pyenv/versions/3.10.19/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+----+-------------------+-----+--------------------+
|type|           duration|event|                   x|
+----+-------------------+-----+--------------------+
|   A|0.09784204010016541|    1|  0.2967393780454473|
|   A|  4.843488321644426|    1| -0.2987735866784475|
|   A|  7.196560508319127|    1|-0.04017394160294...|
|   B|  6.892499338292188|    1| 0.20659237721603418|
|   B|  26.75843017452848|    1|-0.08396970353317602|
+----+-------------------+-----+--------------------+
only showing top 5 rows



## 3. Обучение и продление baseline

In [4]:
model = SparkCoxPHByType(
    type_col='type',
    duration_col='duration',
    event_col='event',
    feature_cols=['x'],
    seed=42,
)
model.fit(sdf)
model.extend_baselines(max_time=25)
model.artifacts

{'A': TypeArtifacts(type_value='A', beta={'x': -0.17203093802722835}, mean_={'x': -0.20381435735001205}, baseline_survival=[1.0, 0.7487158764901778, 0.7007297058019774, 0.6139382135452179, 0.595255199804578, 0.4994902588966358, 0.46061099374514286, 0.40020118113866915, 0.33613803171349604, 0.3133087681088635, 0.2671970461043303, 0.2174604546538099, 0.2174604546538099, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.1579787054082975, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.09201787493452489, 0.02857424632054922], baseline_ratio=[1.0, 0.7487158764901778, 0.9359087042295009, 0.87614126882572, 0.9695685765628532, 0.8391195222832463, 0.9221621153586129, 0.86884852201357

## 4. Прогноз выживаемости

In [5]:
pred = model.predict_survival_at_t(sdf, t=10, output_col='s10')
pred.select('type', 's10').show(5)

+----+-------------------+
|type|                s10|
+----+-------------------+
|   A|0.29793550684686837|
|   A|0.26145173046809583|
|   A| 0.2771673301934135|
|   B|0.27203315658738986|
|   B|0.24349084121407846|
+----+-------------------+
only showing top 5 rows



## 5. Сохранение, загрузка и создание модели из конфига

In [6]:
csv_path = '/tmp/cox_tutorial.csv'
model.save(csv_path)

loaded = SparkCoxPHByType.load(csv_path)
re_pred = loaded.predict_survival_at_t(sdf, t=10, output_col='s10_loaded')
re_pred.select('type', 's10_loaded').show(3)

config_only = SparkCoxPHByType.from_config(csv_path)
config_only.feature_cols

+----+-------------------+
|type|         s10_loaded|
+----+-------------------+
|   A|0.29793550684686837|
|   A|0.26145173046809583|
|   A| 0.2771673301934135|
+----+-------------------+
only showing top 3 rows



['x']

## 6. Останавливаем Spark

In [7]:
spark.stop()